In [1]:
import pandas as pd
import requests
import geoviews as gv
import hvplot.pandas
from geoviews import opts
import geoviews.tile_sources as gvts

# Geoapify API Key
geoapify_key = "4e3524f21e114e50a9ca823f33fc1434"


In [2]:
# Step 1: Load the weather data
weather_df = pd.read_csv("Starter_Code/weather_data.csv")

# Display the first few rows of the data
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,bethel,41.3712,-73.4140,27.90,40,0,3.09
1,stanley,54.8680,-1.6985,8.31,74,0,2.06
2,tindouf,27.6711,-8.1474,26.50,23,93,3.42
3,margaret river,-33.9500,115.0667,12.53,68,72,2.89
4,luang prabang,19.8856,102.1347,21.89,90,26,0.51


In [3]:
# Step 2: Filter for ideal vacation cities

# Ideal conditions: Temperature between 21°C and 27°C, Wind Speed < 4.5 m/s, Cloudiness = 0
ideal_cities = weather_df[
    (weather_df["Temperature"] >= 21) & (weather_df["Temperature"] <= 27) &
    (weather_df["Wind Speed"] < 4.5) & 
    (weather_df["Cloudiness"] == 0)
]

# Display the filtered ideal cities
ideal_cities.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
30,fomboni,-12.2800,43.7425,25.77,73,0,2.36
44,lihue,21.9789,-159.3672,23.61,88,0,3.60
45,wailua homesteads,22.0669,-159.3780,22.90,92,0,3.60
181,nakur,29.9167,77.3000,22.24,52,0,1.58
250,port hedland,-20.3167,118.5667,25.03,78,0,4.12


In [4]:
# Step 3: Use Geoapify API to find the nearest hotel for each ideal city

# Create a list to store hotel data
hotels_list = []

# Iterate over each city in the ideal cities DataFrame
for index, row in ideal_cities.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    city = row["City"]
    country = row.get("Country", "Unknown")  # Assuming there's a Country column in the CSV
    
    # Geoapify URL to find nearby hotels
    query_url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{lon},{lat},10000&limit=1&apiKey={geoapify_key}"
    
    # Make the API request
    hotel_response = requests.get(query_url).json()
    
    # Extract hotel information if available
    try:
        hotel_name = hotel_response["features"][0]["properties"]["name"]
        hotels_list.append({
            "City": city,
            "Country": country,
            "Latitude": lat,
            "Longitude": lon,
            "Hotel Name": hotel_name
        })
    except (IndexError, KeyError):
        # No hotel found
        print(f"No hotel found for {city}.")
        continue

# Convert the list of hotel data into a DataFrame
hotel_df = pd.DataFrame(hotels_list)

# Display the hotel DataFrame
hotel_df.head()


No hotel found for nakur.
No hotel found for dalmau.


,City,Country,Latitude,Longitude,Hotel Name
0,fomboni,Unknown,-12.2800,43.7425,Mohéli La Plage
1,lihue,Unknown,21.9789,-159.3672,Sheraton Kauai Coconut Beach Resort
2,wailua homesteads,Unknown,22.0669,-159.3780,Sheraton Kauai Coconut Beach Resort
3,port hedland,Unknown,-20.3167,118.5667,Hospitality Port Hedland
4,pasni,Unknown,25.2631,63.4710,Juddi Hotel


In [5]:
# Step 4: Create a map with hotel information

# Create a GeoDataFrame with hotel data
hotel_geo_df = gv.Points(
    hotel_df, 
    kdims=["Longitude", "Latitude"], 
    vdims=["City", "Hotel Name"]
)

# Define the tile source for the map (using OpenStreetMap)
tiles = gvts.OSM

# Create a map with hover information for city and hotel names
hotel_map = tiles * hotel_geo_df.opts(
    opts.Points(size=10, color="blue", tools=["hover"], width=800, height=600)
)

# Display the map with hover functionality
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Hotel Name)